<div class="head0">
    <div class="head0__name">
        Basic examples for package
    </div>
    <div class="head0__note">
        Solution of the multidimensional Fokker-Planck equation by fast and accurate tensor based methods.
    </div>
</div>

In [1]:
import sys
import time

import numpy as np

sys.path.extend(['./../lib', './../helpers'])
from intertrain import Intertrain
from solver import Solver as Solver

from helpers import init_jupyter; init_jupyter()

Start | 11:53AM MSK on Sep 04, 2019 |
-------------------------------------


<div class="head1">
    <div class="head1__name">
        Solution strategy
    </div>
</div>

We are going to find probability density function (PDF) $\rho(x, t)$ for the spatial $d$-dimensional ($d \ge 1$) variable $x \in R^d$ which is the solution of the SDE with Brownian motion $\beta$ of dimension $q$ ($q \ge 1$, and we assume below that $q = d$)
$$
    dx = f(x, t) \, dt + S(x, t) \, d\beta,
    \quad
    d\beta \, d\beta^{\top} = Q(t) dt,
    \quad
    x(0) = x_0 \sim \rho(x, 0) = \rho_0 (x),
$$
where $f(x, t) \in R^d$ is a vector-function and $S(x, t) \in R^{d \times q}$ is a matrix-function.

It can be shown that the PDF $\rho(x, t)$ at time $t$ ($t > 0$) is the solution of the Fokker-Planck equation
$$
    \frac{\partial \rho(x, t)}{\partial t} =
        - \sum_{i=1}^d
            \frac{\partial}{\partial x_i}
            \left[ f_i(x, t) \rho(x, t) \right]
        + \sum_{i=1}^d \sum_{j=1}^d
            \frac{\partial^2}{\partial x_i \partial x_j}
            \left[ D_{ij}(x, t) \rho(x, t) \right],
    \quad
    \rho(x, 0) = \rho_0(x),
$$
where $D(x, t)$ is a diffusion tensor of the form
$$
    D(x, t) = \frac{1}{2} S(x, t) Q S^{\top}(x, t).
$$

For simplicity we set $Q(t) \equiv 2 I$, $S(x, t) \equiv I$ (hence $D(x, t) \equiv I$) and then come to the following equations
$$
    dx = f(x, t) \, dt + d\beta,
    \quad
    d\beta \, d\beta^{\top} = 2 I dt,
    \quad
    \frac{\partial \rho}{\partial t} = \Delta \rho - div \left[ f(x, t) \rho \right],
    \quad
    \rho(x, 0) = \rho_0(x).
$$

Let apply 1st order splitting scheme on the $(k+1)$-th time step $kh$ (with known from the previous step $\rho_{k}$)
$$
\frac{\partial v}{\partial t} = \Delta v,
\quad
v_{k} = \rho_{k},
$$
$$
\frac{\partial w}{\partial t} = - div \left[ f(x, t) w \right],
\quad
w_{k} = v_{k+1},
$$
then we can approximate a value of the PDE $\rho$ at time step $k+1$ as $\rho_{k+1} = w_{k+1}$.

Note that equation for $w$ may be refolmulated in terms of the trajectory integration:
$$
    \begin{cases}
        \frac{\partial \, x}{\partial \, t} = f(x, t) \\
        \frac{\partial \, w}{\partial \, t} =
            -tr \left[ \frac{\partial \, f}{\partial \, x} (x, t) \right] w
    \end{cases},
$$
or equivalently
$$
    \begin{cases}
        \frac{\partial \, x}{\partial \, t} = f(x, t) \\
        \frac{\partial \, \log{w}}{\partial \, t} =
            -tr \left[ \frac{\partial \, f}{\partial \, x} (x, t) \right]
    \end{cases},
$$
but for such kind of equations we have to interpolate PDF on each time step to obtain its nodal values.

<div class="head1">
    <div class="head1__name">
        Algorithm (1th order)
    </div>
    <div class="head1__note">
        This is a draft of the algorithm. which is used in solver module/class (Russian is used for simplicity).
    </div>
</div>

**Задача:** найти PDF $\rho$ в момент времени $t$ на Чебышевской сетке из уравнений
$$
    dx = f(x, t) \, dt + d\beta,
    \quad
    d\beta \, d\beta^{\top} = 2 I dt,
    \quad
    \frac{\partial \rho}{\partial t} = \Delta \rho - div \left[ f(x, t) \rho \right],
    \quad
    \rho(x, 0) = \rho_0(x).
$$

**Нужно** передать в крест (tt-cross) функцию (step), которая вычисляет значения $\rho_{k+1}$ в заданном (произвольном) наборе точек $X$ Чебышевской сетки. Это позволит проинтерполировать $\rho$ на $(k+1)$-ом шаге и перейти к следующему шагу, при этом предполагается, что интерполянт для $k$-ого шага уже известен.

> На нулевом шаге имеем $\rho_0(x)$, заданную как функцию от $x$ и можем построить интерполянт очевидным образом.

> Если не используем крест (работаем в полном numpy формате), то $X$ - это полный набор точек Чебышевской сетки (алгоритм при этом остается прежним).

> Алгоритм расписан для splitting схемы первого порядка, поэтому для решения ODE можем использовать метод Эйлера без потери точности.

**Работа функции step(X)**

> X - произвольный набор точек Чебышевской сетки (ndarray [dims, pois] of float)

**1** Найти прообразы $\widehat{X}$ (соответствуют предыдущему $k$-ому шагу) для заданного набора точек $X$ Чебышевской сетки, которые приводили бы траекторию детерминированного уравнения ($\beta = 0$) в точки $X$ на $(k+1)$-ом шаге, интегрируя назад уравнение
$$
    \frac{\partial \, x}{\partial \, t} = f(x, t),
    \quad
    x_{k+1} = X,
    \quad
    x_{k} = \widehat{X} = ?,
$$
используя формулу Эйлера
$$
    \widehat{X} = X - h \cdot f(X, t_{k+1}).
$$

**2** Используя известный интерполянт $E^{(\rho)}_k$ на $k$-ом шаге, вычислить в точках $\widehat{X}$ значения PDF $\widehat{\rho} = \tilde{E}^{(\rho)}_k(\widehat{X})$, где знак ~ в $\tilde{E}^{(\rho)}_k$ соответствует обнулению значений для точек, которые оказались вне пределов интерполяции.

**3** Решить PDE с однородными граничными условиями Дирихле для $(k+1)$-ого шага
$$
    \frac{\partial v}{\partial t} = \Delta v,
    \quad
    v_{k} = \widehat{\rho},
    \quad
    v_{k+1} = v = ?,
$$
используя дифференциальную матрицу Чебышева 2-ого порядка $D$
$$
    v = 
        \left( e^{\sqrt[d]{h} J D} \otimes \ldots \otimes e^{\sqrt[d]{h} J D} \right)
        \left( J \otimes \ldots \otimes J \right)
        \widehat{\rho},
$$
где матрица $J = diag \left[ 0 \, 1 \, \ldots \, 1 \, 0 \right]$ введена для удовлетворения однородным граничным условиям.

**4** Решить ODE для $k+1$-ого шага
$$
    \frac{\partial \, w}{\partial \, t} =
        -tr \left[ \frac{\partial \, f}{\partial \, x} (x, t) \right] w,
    \quad
    w_{k} = v,
    \quad
    w_{k+1} = w = ?,
$$
при условии интегрирования вдоль траектории
$$
    \frac{\partial \, x}{\partial \, t} = f(x, t),
    \quad
    x_{k} = \widehat{X},
$$
(приводящей, в соответствии с пунктом 1 в $x_{k+1} = X$), используя формулу Эйлера
$$
    w = \left(
        1 - h \cdot tr \left[ \frac{\partial \, f}{\partial \, x}(\widehat{X}, t_{k}) \right]
    \right) v.
$$

**5** Вернуть значение $w$ как приближение $\rho(x)$ в заданном (произвольном) наборе точек $X$ Чебышевской сетки на $(k+1)$-ом шаге.

**Примечание** На 4-ом шаге в качестве альтернативного варианта может быть решено эквивалентное уравнение
$$
    \frac{\partial \, \log{w}}{\partial \, t} =
        -tr \left[ \frac{\partial \, f}{\partial \, x} (x, t) \right],
    \quad
    w_{k} = v,
    \quad
    w_{k+1} = w = ?.
$$

<div class="head1">
    <div class="head1__name">
        Short version of the Algorithm (1th order)
    </div>
    <div class="head1__note">
        This is a draft of the algorithm. which is used in solver module/class (Russian is used for simplicity).
    </div>
</div>

В качестве итоговой формулы (первого порядка точности) для вычисления значения $\rho$ на $(k+1)$-ом временном шаге в заданном (произвольном) наборе точек $X$ Чебышевской сетки по известному интерполянту $E^{(\rho)}_k$ для $\rho$ на $k$-ом шаге получили
$$
    \widehat{X} = X - h \cdot f(X, t_{k+1}),
$$
$$
    \rho_{k+1}(X) =
        \left(
            1 - h \cdot tr \left[ \frac{\partial \, f}{\partial \, x}(\widehat{X}, t_{k}) \right]
        \right)
        \left( e^{\sqrt[d]{h} J D} \otimes \ldots \otimes e^{\sqrt[d]{h} J D} \right)
        \left( J \otimes \ldots \otimes J \right)
        \tilde{E}^{(\rho)}_k(\widehat{X}),
$$
где знак ~ в $\tilde{E}^{(\rho)}_k$ соответствует обнулению значений для точек, которые оказались вне пределов интерполяции, $D$ - дифференциальная матрица Чебышева 2-ого порядка, а матрица $J = diag \left[ 0 \, 1 \, \ldots \, 1 \, 0 \right]$ введена для удовлетворения однородным граничным условиям.

<div class="head1">
    <div class="head1__name">
        Solution of the model problem
    </div>
    <div class="head1__note">
        Numerical example with detailed comments about solver API will be here...
    </div>
</div>



<div class="end"></div>